In [1]:
import os
import torch

import os
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn


sys.path.append('..')
sys.path.append('.')
sys.path.append('../llm_garden')

from peft_modules import peft_utils
import esm_adapter



In [2]:
torch.hub.set_dir("/home/zengs/zengs_data/torch_hub")

In [3]:

INPUT_SEQ_SIZE = 50
data = peft_utils.get_esm_example_data(INPUT_SEQ_SIZE)

model, alphabet = esm_adapter.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

batch_labels, batch_strs, batch_tokens = batch_converter(data)

true_seqs = [ item[1] for item in data ]

In [4]:
logits = model(batch_tokens)["logits"]
tokens = torch.argmax(logits, dim=2)

pred_seqs = []
for one_sample in tokens:
  one_sample = one_sample[1:-1]
  one_seq = "".join([model.alphabet.get_tok(one_token) for one_token in one_sample])
  pred_seqs.append(one_seq)


In [5]:

def calc_acc(pred_seq, true_seq):
  acc = 0
  for pred_residue, true_residue in zip(pred_seq, true_seq):
    if pred_residue == true_residue:
      acc += 1
  return acc / len(pred_seq)

acc = [calc_acc(pred_seq, true_seq) for pred_seq, true_seq in zip(pred_seqs, true_seqs) ]


print(true_seqs)
print(pred_seqs)
print(acc)



['AKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLG', 'BALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALA', 'DKEVFDLIRDEEIRHLKALARKQISQTGMPPTRAEIAQRLGFRSPNAAEH', 'ERDHISQTGMPPTRAEIAQRLDHISQTGMPPTRRDEEIRHLLKEVFDLIR']
['MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLG', 'MALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALA', 'MKEVFDLIRDEEIRHLKALARKQISQTGMPPTRAEIAQRLGFRSPNAAEH', 'MRDHISQTGMPRTRAEIAQRLDHISQTGMPPTRTDEEIRHLLKEVFDLIR']
[0.98, 0.98, 0.98, 0.94]
